In [166]:
import os
from dotenv import load_dotenv
import pandas as pd

# Load environment variables from .env file
load_dotenv()

# Get API key from environment variable
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')


In [167]:
from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)

In [185]:
model_name = "gemini-2.5-pro-preview-05-06" # @param ["gemini-1.5-flash-latest","gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-flash-preview-04-17","gemini-2.5-pro-exp-03-25"] {"allow-input":true, isTemplate: true}

In [177]:
df_trial_fp = pd.read_csv("/Volumes/T7/OMSCS/CLEF2025/EXIST2025/exist-2025/notebooks/gemini/eval_results/trial1_fp.csv")
df_trial_fp['label_num'] = df_trial_fp['label'].map({'YES': 1.0, 'NO': 0.0})
# df_trial_fp = df_trial_fp[:10]
df_trial_fp = df_trial_fp.sample(n=4)

df_trial_fp

,Unnamed: 0,id_EXIST,text,path_video,target,description,label,label_num
139,744,220482,feel free to put ur opinions on the comments |...,videos/6978303589928291589.mp4,0.0,The video guesses viewers' sexual orientations...,YES,1.0
42,247,220840,this is most men's clit. so i'm,videos/7099866647367781674.mp4,0.0,A TikTok video juxtaposes a woman reacting wit...,YES,1.0
117,663,220548,am i right or am i right? passport united stat...,videos/6994831286594587909.mp4,0.0,A woman uses a derogatory term to criticize ot...,YES,1.0
31,180,220487,"reply to still gonna rock all of these, would...",videos/6979219805903621382.mp4,0.0,A woman humorously critiques the revealing and...,YES,1.0


In [178]:
train_df = pd.read_csv("/Volumes/T7/OMSCS/CLEF2025/EXIST2025/exist-2025/notebooks/train_test_split/train_df.csv")
valid_df = pd.read_csv("/Volumes/T7/OMSCS/CLEF2025/EXIST2025/exist-2025/notebooks/train_test_split/valid_df.csv")
test_df = pd.read_csv("/Volumes/T7/OMSCS/CLEF2025/EXIST2025/exist-2025/notebooks/train_test_split/test_df.csv")

cols_to_rem = ['id_Tiktok','lang','video','url','annotators',
               'number_annotators','gender_annotators',
               'labels_task3_1','labels_task3_2','labels_task3_3','split']

train_df = train_df.drop(columns=cols_to_rem)
valid_df = valid_df.drop(columns=cols_to_rem)
test_df = test_df.drop(columns=cols_to_rem)

df_all = pd.concat([train_df, valid_df, test_df], axis=0, ignore_index=True)
df_all


,id_EXIST,text,path_video,target
0,220242,these men stay sparkles pressed sparkles che...,videos/6920327322679692545.mp4,1
1,220296,beaming_face_with_smiling_eyes ‘school’ pent...,videos/6935046770778967302.mp4,1
2,220920,"don’t laugh at your girlfriends choice, you ar...",videos/7123342389338443009.mp4,0
3,220768,i guess she’s not wrong face_with_tears_of_jo...,videos/7076153533996928302.mp4,1
4,220665,"women, why do we do this? woman_facepalming_m...",videos/7039365996053794053.mp4,1
...,...,...,...,...
968,220599,can i come live with someone else pls? clean t...,videos/7014652808725155078.mp4,1
969,220337,"with walking into my purpose, a new challenge ...",videos/6943386660419718406.mp4,0
970,220901,men are being lazy. you see ladies out here g...,videos/7119092344711859462.mp4,1
971,220872,"xin chào officially, there are 42 offices with...",videos/7108633528530701594.mp4,0


In [179]:
# Get the id_EXIST values from df_trial_fp
trial_ids = df_trial_fp['id_EXIST'].values

# Remove rows from df_all where id_EXIST is in trial_ids
df_all = df_all[~df_all['id_EXIST'].isin(trial_ids)]
n =10
# Get equal number of samples from each target class
n_per_class = n // 2  # Integer division to get equal numbers
df_0 = df_all[df_all['target'] == 0].sample(n=n_per_class, replace=True)
df_1 = df_all[df_all['target'] == 1].sample(n=n_per_class + (n % 2), replace=True)  # Add remainder if odd
df_all = pd.concat([df_0, df_1])

df_all

,id_EXIST,text,path_video,target
963,220601,text maddison hutton. clean the above detect...,videos/7014944407195569413.mp4,0
459,220033,"having the best mom be like... step-mom, you c...",videos/6814244764108541190.mp4,0
548,220303,she’s so cute when she’s sleepy pleading_face ...,videos/6938076074215542021.mp4,0
378,220224,find a pregnant lady and hold her belly 'daddy...,videos/6913316057935514885.mp4,0
31,220255,i don’t like my step mom at all. dad can i pl...,videos/6923864192139152645.mp4,0
314,220671,"men and women are often compared, but the com...",videos/7041569616254012677.mp4,1
365,220402,"girls vs boys girls playing forza at 3am ""com...",videos/6962382682248482053.mp4,1
601,220396,"lower your expectations! you want a nice girl,...",videos/6961557595891043589.mp4,1
584,220250,the last one is so important. which is your fa...,videos/6922184169908915462.mp4,1
921,220463,reply to i still might buy that cyber truck ...,videos/6973835716358917382.mp4,1


In [180]:
# Select only the specified columns from both dataframes
df_all_subset = df_all[['id_EXIST', 'path_video', 'target','text']]
df_trial_subset = df_trial_fp[['id_EXIST', 'path_video', 'target','text']]

# Concatenate the two dataframes
df_combined = pd.concat([df_all_subset, df_trial_subset], axis=0, ignore_index=True)
df_combined

,id_EXIST,path_video,target,text
0,220601,videos/7014944407195569413.mp4,0.0,text maddison hutton. clean the above detect...
1,220033,videos/6814244764108541190.mp4,0.0,"having the best mom be like... step-mom, you c..."
2,220303,videos/6938076074215542021.mp4,0.0,she’s so cute when she’s sleepy pleading_face ...
3,220224,videos/6913316057935514885.mp4,0.0,find a pregnant lady and hold her belly 'daddy...
4,220255,videos/6923864192139152645.mp4,0.0,i don’t like my step mom at all. dad can i pl...
5,220671,videos/7041569616254012677.mp4,1.0,"men and women are often compared, but the com..."
6,220402,videos/6962382682248482053.mp4,1.0,"girls vs boys girls playing forza at 3am ""com..."
7,220396,videos/6961557595891043589.mp4,1.0,"lower your expectations! you want a nice girl,..."
8,220250,videos/6922184169908915462.mp4,1.0,the last one is so important. which is your fa...
9,220463,videos/6973835716358917382.mp4,1.0,reply to i still might buy that cyber truck ...


In [181]:
# sexism_prompt = (
#     "You are an expert in detecting sexism in video content. Take the role of a female"
#     "Your task is to assess whether the video either displays or critiques sexist content. "
#     "You will first be shown several example videos that have already been labeled as either sexist or not sexist, based on clear criteria."
#     "Sexism is considered present if the content falls into any of the following five categories: "
#     "(i) IDEOLOGICAL AND INEQUALITY — promoting or reinforcing beliefs about male superiority or social inequality between genders, "
#     "(ii) STEREOTYPING AND DOMINANCE — reinforcing traditional gender roles or power imbalance, "
#     "(iii) OBJECTIFICATION — reducing a person (typically a woman) to their body or physical appearance, "
#     "(iv) SEXUAL VIOLENCE — explicit or implied depictions of harassment, assault, or coercion, "
#     "(v) MISOGYNY AND NON-SEXUAL VIOLENCE — expressing hatred or aggression toward women in non-sexual ways. "
#     "You must be **extra critical and cautious** in your assessment. Ultimately, the judgement criteria used in the example videos is the one you should use to label the new videos."
#     "Please respond strictly in the following JSON format:\n\n"
#     '{\n'
#     '  "description": "[Your one-sentence description of the video]",\n'
#     '  "label": "YES" or "NO",\n'
#     '  "analysis": "[One or two sentences explaining why the video is or isnt sexist, with reference to one or more of the five categories]"\n'
#     '}\n\n'
#     "Only return valid JSON. Do not include any explanations, category names, or extra text outside the JSON."
# )

sexism_prompt = (
    """
    You are an expert in identifying the presence of sexism in video content. 
    Your task is to assess whether the video shows or critiques sexism (defined as marginalization, stereotypes about women, male superiority, or misogyny). 
    {
    "description": "[One sentence describing the video's relevant content AND, crucially, the video's *own apparent stance or framing* of that content, focusing on whether the video *itself* promotes, endorses, or normalizes sexism.]",
    "label": "YES" or "NO"
    "analysis": [One sentence describing the reason why it was labelled as YES or NO]
    "probability": Give a number between 0 and 1 based on how confident you are on the label where >= 0.5 and < 1 implies a label of "YES" and between 0 and 0.5 implies a label of NO
    }
    Only return valid JSON. Do not include any explanations or extra text.
    """
)

# sexism_prompt = (
#     """
#     You are an expert in identifying sexism in video content. Your primary task is to distinguish between videos that *are themselves sexist* (i.e., they actively promote, endorse, or normalize sexism through their own message, framing, or overall presentation) and videos that merely *depict or discuss* sexist content (e.g., for critique, satire, factual reporting, or as part of a narrative without the video *itself* endorsing the sexism).

#     **The core question is: Does the video *itself* actively promote, endorse, or normalize sexism?**

#     Sexism is defined as the *video itself* communicating one or more of the following, in a way that the video *actively promotes or clearly endorses as acceptable, normal, or true*, thereby contributing to inequality or harm:

#     *   **Marginalization:** The video *itself* presents women as inherently less important, capable, or deserving, or unfairly limits their opportunities or voice *due to their gender*, in a manner the video appears to support or normalize.
#     *   **Harmful Stereotypes:** The video *itself* actively promotes demeaning, limiting, or broadly negative generalizations about women that reinforce inequality. This excludes neutral or humorous gender-based observations *unless the video clearly uses that humor with the primary intent to belittle, restrict women, or justify inequality based on their gender, and the video's overall framing endorses this harmful impact.*
#     *   **Male Superiority:** The video *itself* asserts or implies men's inherent dominance, greater value, or rightful control over women, framing this as natural, desirable, or justified.
#     *   **Misogyny:** The video *itself* expresses or encourages hatred, contempt, or ingrained prejudice against women.

#     **Crucial Decision Point (Labeling):**

#     *   **Label "YES"**: If the video's *own discernible message, tone, narrative voice, or overall presentation* clearly and actively promotes, endorses, normalizes, or celebrates any of the sexist elements defined above. The video itself is the source of sexist advocacy or validation.
#     *   **Label "NO"**:
#         *   If the video depicts sexist acts, language, or ideas *primarily to critique, condemn, satirize, or factually report on them*, where the video's own stance is clearly against the depicted sexism or unendorsing of it.
#         *   If sexist ideas/actions are expressed by characters or are part of a narrative, but the *video itself does not demonstrably endorse or promote these as valid, acceptable, or desirable*. The video might be exploring complex themes or showing flawed characters without its own voice condoning the sexism.
#         *   If sexist elements are merely incidental (e.g., background elements not central to any message actively endorsed by the video) and are not the focus of the video's *own* active promotion or endorsement.
#         *   If humor touches on gender, but the video's *primary discernible intent* is not to demean a gender or promote inequality, and the humor doesn't rely on the video endorsing harmful sexist premises for its effect.

#     Respond strictly in the following JSON format:
#     {
#     "description": "[One sentence describing the video's relevant content AND, crucially, the video's *own apparent stance or framing* of that content, focusing on whether the video *itself* promotes, endorses, or normalizes sexism.]",
#     "label": "YES" or "NO"
#     "analysis": [One sentence describing the reason why it was labelled as YES or NO]
#     "probability": Give a number between 0 and 1 based on how confident you are on the label where >= 0.5 and < 1 implies a label of "YES" and between 0 and 0.5 implies a label of NO
#     }

#     Only return valid JSON. Do not include any explanations or extra text.
#     """
# )

In [182]:
print(sexism_prompt)


    You are an expert in identifying the presence of sexism in video content. 
    Your task is to assess whether the video shows or critiques sexism (defined as marginalization, stereotypes about women, male superiority, or misogyny). 
    {
    "description": "[One sentence describing the video's relevant content AND, crucially, the video's *own apparent stance or framing* of that content, focusing on whether the video *itself* promotes, endorses, or normalizes sexism.]",
    "label": "YES" or "NO"
    "analysis": [One sentence describing the reason why it was labelled as YES or NO]
    "probability": Give a number between 0 and 1 based on how confident you are on the label where >= 0.5 and < 1 implies a label of "YES" and between 0 and 0.5 implies a label of NO
    }
    Only return valid JSON. Do not include any explanations or extra text.
    


In [183]:
import time

def upload_video(video_file_name):
    video_file = client.files.upload(file=video_file_name)

    while video_file.state == "PROCESSING":
        print(f'Processing {video_file_name}...')
        time.sleep(10)
        video_file = client.files.get(name=video_file.name)

    if video_file.state == "FAILED":
        raise ValueError(f'Video processing failed for {video_file_name}')
    
    print(f'Video processing complete: {video_file.uri}')
    return video_file

In [184]:
import json
import pandas as pd
from tqdm import tqdm

# Settings
max_iterations = 5
target_accuracy = 0.90

# Logging
prompt_versions = [sexism_prompt]
accuracy_log = []

df = df_combined

def run_model_on_dataframe(df_input, prompt, client, model_name):
    descriptions = []
    labels = []
    analysiss = []
    prob = []

    for idx, row in tqdm(df_input.iterrows(), total=len(df_input)):
        try:
            path = f"/Users/moiz.ali/Downloads/EXIST 2025 Dataset V0.3/EXIST 2025 Videos Dataset/training/{row['path_video']}"
            uploaded_video = upload_video(path)

            response = client.models.generate_content(
                model=model_name,
                contents=[uploaded_video, prompt]
            )
            result = json.loads(response.text.strip().strip("```json").strip("```"))

            descriptions.append(result.get("description", "ERROR"))
            labels.append(result.get("label", "ERROR"))
            analysiss.append(result.get("analysis", "ERROR"))
            prob.append(result.get("probability", "ERROR"))

        except Exception as e:
            print(f"❌ Error at index {idx} (id {row['id_EXIST']}): {e}")
            descriptions.append("ERROR")
            labels.append("ERROR")
            analysiss.append("ERROR")
            prob.append("ERROR")

    df_input["description"] = descriptions
    df_input["label"] = labels
    df_input["analysis"] = analysiss
    df_input["probability"] = prob

    return df_input


def evaluate_predictions(df_eval):
    correct = 0
    errors = []

    for _, row in df_eval.iterrows():
        if row["label_num"] == row["target"]:
            correct += 1
        else:
            errors.append({
                "id": row["id_EXIST"],
                "text": row["text"],
                "expected": row["target"],
                "predicted": row["label"],
                "description": row["description"],
                "analysis": row["analysis"],
                "probability": row["probability"],
            })

    accuracy = correct / len(df_eval)
    return accuracy, errors


def refine_prompt(current_prompt, errors, client, model_name):
    example_text = "\n\n".join([
        f"Video transcript: {e['text']}\nExpected label: {e['expected']}\nPredicted: {e['predicted']}\nGemini's description: {e['description']}\nGemini's analysis: {e['analysis']}\nGemini's label probability: {e['probability']}"
        for e in errors[:10]
    ])

    refinement_prompt = f"""
    You are helping refine a prompt that instructs an AI to classify whether a video is sexist or not.

    Here's the current prompt:
    ---
    {current_prompt}
    ---

    The AI was tested on some examples and misclassified the following:
    {example_text}

    Based on these errors, please rewrite or revise the original prompt to help the AI better distinguish between:
    - when a video critiques vs displays sexism
    - how to apply the 5 categories more effectively
    - how to reduce false positives and false negatives
    - keep the prompt as concise as possible

    Output ONLY the new prompt. No other explanation.
    """

    print(example_text)

    try:
        refinement_response = client.models.generate_content(
            model=model_name,
            contents=[refinement_prompt]
        )
        new_prompt = refinement_response.text.strip()
        return new_prompt
    except Exception as e:
        print(f"⚠️ Prompt refinement failed: {e}")
        return current_prompt


# Main loop
iteration = 0
df_current = df.copy()

while iteration < max_iterations:
    print(f"\n🚀 Iteration {iteration + 1}")

    # Run model
    df_current = run_model_on_dataframe(df_current, sexism_prompt, client, model_name)
    print(df_current)
    df_current['label_num'] = df_current['label'].map({'YES': 1.0, 'NO': 0.0})

    # Evaluate
    accuracy, errors = evaluate_predictions(df_current)
    accuracy_log.append(accuracy)
    print(f"📊 Accuracy at iteration {iteration + 1}: {accuracy:.2%}")

    # Save full output for this iteration
    df_current.to_csv(f"results_iteration_{iteration+1}.csv", index=False)

    if accuracy >= target_accuracy:
        print("✅ Target accuracy reached.")
        break

    if not errors:
        print("⚠️ No errors found, but accuracy is still low. Stopping.")
        break

    # Refine prompt using incorrect examples
    sexism_prompt = refine_prompt(sexism_prompt, errors, client, model_name)
    print(f"New prompt: {sexism_prompt}")
    prompt_versions.append(sexism_prompt)

    # Create new df with only misclassified rows
    # error_ids = {e['id'] for e in errors}
    # df_current = df[df['id_EXIST'].isin(error_ids)].copy()

    iteration += 1

# Save final logs
with open("prompt_versions.txt", "w") as f:
    for i, prompt in enumerate(prompt_versions):
        f.write(f"\n\n--- Prompt Version {i+1} ---\n{prompt}\n")

with open("accuracy_log.json", "w") as f:
    json.dump(accuracy_log, f, indent=2)

print("🏁 Finished.")



🚀 Iteration 1


  0%|          | 0/14 [00:00<?, ?it/s]

Processing /Users/moiz.ali/Downloads/EXIST 2025 Dataset V0.3/EXIST 2025 Videos Dataset/training/videos/7014944407195569413.mp4...
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/b9gt4c8whbr1


  7%|▋         | 1/14 [00:21<04:36, 21.23s/it]

Processing /Users/moiz.ali/Downloads/EXIST 2025 Dataset V0.3/EXIST 2025 Videos Dataset/training/videos/6814244764108541190.mp4...
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/dmo3dwrpoxdb


...

 93%|█████████▎| 13/14 [09:53<00:42, 42.87s/it]

Processing /Users/moiz.ali/Downloads/EXIST 2025 Dataset V0.3/EXIST 2025 Videos Dataset/training/videos/6979219805903621382.mp4...
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/k9a7mojbdojq


100%|██████████| 14/14 [10:24<00:00, 44.59s/it]


    id_EXIST                      path_video  target  \
0     220601  videos/7014944407195569413.mp4     0.0   
1     220033  videos/6814244764108541190.mp4     0.0   
2     220303  videos/6938076074215542021.mp4     0.0   
3     220224  videos/6913316057935514885.mp4     0.0   
4     220255  videos/6923864192139152645.mp4     0.0   
5     220671  videos/7041569616254012677.mp4     1.0   
6     220402  videos/6962382682248482053.mp4     1.0   
7     220396  videos/6961557595891043589.mp4     1.0   
8     220250  videos/6922184169908915462.mp4     1.0   
9     220463  videos/6973835716358917382.mp4     1.0   
10    220482  videos/6978303589928291589.mp4     0.0   
11    220840  videos/7099866647367781674.mp4     0.0   
12    220548  videos/6994831286594587909.mp4     0.0   
13    220487  videos/6979219805903621382.mp4     0.0   

                                                 text  \
0    text  maddison hutton. clean the above detect...   
1   having the best mom be like... step-mom, 

KeyboardInterrupt: 